# Image-to-Image Transformations

We can also use our pipelines to take existing images and use them as inspiration to generate new images.

In [ ]:
# @title Import Libraries and Set Everything Up

import matplotlib.pyplot as plt
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline
from google.colab import output
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Clear CUDA cache to free up GPU memory, just in case we run this multiple times.
if torch.cuda.is_available():
    torch.cuda.empty_cache()

We're going to use two different pipelines for this:

- `StableDiffusionPipeline`: for generating an image from a text prompt.
- `StableDiffusionImg2ImgPipeline`: for modifying an existing image based on a new prompt and a strength parameter.

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"

# Load the text-to-image pipeline
text_to_image_pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
).to(device)

# Load the image-to-image pipeline
img_to_img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
).to(device)

In [ ]:
base_prompt = "a simple sketch of a house" # @param {type:"string"}
transform_prompt = "a futuristic glass house with neon lights, cyberpunk style" # @param {type:"string"}

The second pipeline has the model weights, but a different pipeline—this one accepts an existing image and transforms it based on a prompt and strength setting. Let's generate an image to start out with.

In [ ]:
base_image = text_to_image_pipe(base_prompt).images[0]

base_image

I'm going to set up a quick diagram so that we can see the results at various differents strengths.

In [ ]:
strengths = [0.3, 0.5, 0.7]

# Set up a plot so we can see all of the different strengths.
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Show original
axes[0].imshow(base_image)
axes[0].set_title("Original")
axes[0].axis('off')

# Generate a new image a each of the different strengths.
for i, strength in enumerate(strengths):
    print(f"Transforming with strength {strength}…")
    transformed = img_to_img_pipe(
        prompt=transform_prompt,
        image=base_image,
        strength=strength,
        guidance_scale=7.5
    ).images[0]

    axes[i+1].imshow(transformed)
    axes[i+1].set_title(f"Strength: {strength}")
    axes[i+1].axis('off')

output.clear()

plt.suptitle("Image-to-Image Transformation", fontsize=16)
plt.tight_layout()
plt.show()